In [ ]:
#|default_exp rag

# ShellSage Retrieval Augmented Generation

## Imports

In [ ]:
from chonkie import SentenceChunker
from fastcore.all import *
from lancedb import connect
from lancedb.pydantic import LanceModel, Vector
from pathlib import Path
from sentence_transformers import SentenceTransformer
from sentence_transformers.models import StaticEmbedding
from subprocess import check_output as co

import re, subprocess

In [ ]:
import os, random
os.environ['TOKENIZERS_PARALLELISM'] = 'false'

In [ ]:
#| export
# set up db path in user's home directory
db_path = Path.home() / "shell_sage" / "db"
db_path.mkdir(parents=True, exist_ok=True)

In [ ]:
#| export
chunker = SentenceChunker(tokenizer="gpt2", chunk_size=2_048,
                          chunk_overlap=256, min_sentences_per_chunk=1)

In [ ]:
#| export
static_embedding = StaticEmbedding.from_model2vec("minishlab/potion-retrieval-32M")
model = SentenceTransformer(modules=[static_embedding], device="cpu")
ndim = model.encode(["Example sentence"]).shape[1]

In [ ]:
ndim

512

In [ ]:
#| export
class EmbeddingTable(LanceModel):
    content: str
    package_name: str
    embedding: Vector(ndim)

In [ ]:
#| export
db = connect(db_path)
tbl = None
try: tbl = db.open_table("man_pages")
except ValueError:
    tbl = db.create_table("man_pages", schema=EmbeddingTable, mode="create")
    tbl.create_fts_index("content") # for hybrid search

In [ ]:
db_path.ls()

(#1) [Path('/Users/nathan/shell_sage/db/man_pages.lance')]

In [ ]:
#| export
def _get_page(cmd):
    try: return cmd, co(['man', cmd], text=True,
                   stderr=subprocess.DEVNULL).strip()
    except: return cmd, None

def clean(text: str) -> str:
    "Cleans man page text by removing backspaces and ANSI escape codes"

    # Remove backspaces
    while True:
        cleaned = re.sub(r'.\x08', '', text)
        if cleaned == text:
            break
        text = cleaned

    # Remove ANSI escape codes
    text = re.sub(r'(?:\x1B[@-Z\\-_]|\x1B\[?[\d;]*[A-PRZcf-nq-uy=><])', '', text)

    # Convert multiple newlines into one
    text = re.sub(r'\n+', '\n', text)
    return text.strip()

def _man_pages(lim=None):
    lines = L(co(['man', '-k', '.'], text=True).strip().splitlines())
    lines = lines.filter(lambda s: re.search(r'\(1[^\)]*\)', s))
    cmds = (lines.map(lambda s: s.split("(")[0].strip())
                 .filter(lambda s: s).unique()[:lim])

    pages = parallel(_get_page, cmds).filter(lambda x: x[1])
    return zip(*pages.map(lambda x: (x[0], clean(x[1]))))


In [ ]:
cmds, pages = _man_pages(16)

In [ ]:
idx = random.randint(0, len(pages))
cmd, page = cmds[idx], pages[idx]
print(cmd)
print(page[:128])

DeRez
DeRez(1)                    General Commands Manual                   DeRez(1)
NAME
       DeRez - decompiles resources (DEPRECA


In [ ]:
#| export
def index_man_pages(cmds, pages):
    for cmd, chunks in zip(cmds, chunker.chunk_batch(pages)):
        embds = model.encode([chunk.text for chunk in chunks])
        tbl.add([EmbeddingTable(content=c.text, package_name=cmd, embedding=emb)
                 for c, emb in zip(chunks, embds)])

In [ ]:
index_man_pages(cmds, pages)

🦛 choooooooooooooooooooonk 100% • 15/15 docs chunked [00:00<00:00, 112.42doc/s] 🌱


In [ ]:
#| export
def search(query):
    q_emb = model.encode([query])
    return tbl.search(q_emb).metric("cosine").limit(2).to_pandas()

In [ ]:
df = search("How can I change my current branch?")
df.head()


,content,package_name,embedding,_distance
0,standard error output if any file...,ar,"[0.695339, 1.6662909, 0.04321767, 0.716739, -0...",0.858607
1,SETFILE(1) General Commands M...,SetFile,"[0.31140828, 1.8759634, -0.709025, 0.59569347,...",0.861011
